In [1]:
import sys
sys.path.append("../../smsl")

In [2]:
import smsl

In [3]:
from smsl.opt import rcsb2charmmformat_sh, charmm2rcsbformat_sh, charmm_c43b1, toppar

### Config Setting

In [4]:
pdbid = "2jpz"

In [5]:
pymol = "/opt/PyMOL-3.0.3_appveyor849-Linux-x86_64-py310/pymol/bin/pymol"

In [6]:
print(f'''\
rcsb2charmmformat_sh = {rcsb2charmmformat_sh}
charmm2rcsbformat_sh = {charmm2rcsbformat_sh}
charmm_c43b1 = {charmm_c43b1}
pymol = {pymol}
''')

rcsb2charmmformat_sh = /home/user/workfolder/g4dna_protocol/smsl/opt/atomname_transfer/rcsb2charmmformat.sh
charmm2rcsbformat_sh = /home/user/workfolder/g4dna_protocol/smsl/opt/atomname_transfer/charmm2rcsbformat.sh
charmm_c43b1 = /home/user/workfolder/g4dna_protocol/smsl/opt/c43b1/bin/charmm
pymol = /opt/PyMOL-3.0.3_appveyor849-Linux-x86_64-py310/pymol/bin/pymol



### 1.Download Protein From RCSB

In [7]:
!wget https://files.rcsb.org/download/{pdbid.upper()}.pdb

--2024-12-18 09:33:39--  https://files.rcsb.org/download/2JPZ.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.245
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘2JPZ.pdb’

2JPZ.pdb                [   <=>              ] 686.84K   712KB/s    in 1.0s    

2024-12-18 09:33:41 (712 KB/s) - ‘2JPZ.pdb’ saved [703323]



In [8]:
!wget https://files.rcsb.org/ligands/download/K.cif

--2024-12-18 09:33:42--  https://files.rcsb.org/ligands/download/K.cif
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.245
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3403 (3.3K) [application/octet-stream]
Saving to: ‘K.cif’

K.cif               100%[===================>]   3.32K  --.-KB/s    in 0s      

2024-12-18 09:33:42 (3.10 GB/s) - ‘K.cif’ saved [3403/3403]



### 2. Preprocess PDB File

In [9]:
pymol_script = f'''\
## 1. load pdb file from rcsb
load {pdbid.upper()}.pdb
## 2. load 2 potassium ions
load K.cif
create K2, K
set_name K, K1
## 3. (optional) optimize the 3D view
set sphere_scale, 0.6
zoom
## 4. move potassium to the central of tetrad
translate [-3.137, 0.213,-0.343], K1
translate [ 0.902,-0.001,-0.020], K2
## 5. define the resid of potassium
alter model K1, resi="27"
alter model K2, resi="28"
## 6. save the molecule of dna and ion part separately to pdb
save {pdbid}_dna.pdb, polymer.nucleic
save {pdbid}_ion.pdb, inorganic
'''
!echo "{pymol_script}" > {pdbid}_setup.pml
!{pymol} -c {pdbid}_setup.pml

 PyMOL(TM) 3.0.3 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
 Detected 12 CPU cores.  Enabled multithreaded rendering.
PyMOL>load 2JPZ.pdb
HEADER    DNA                                     25-MAY-07   2JPZ
TITLE     HUMAN TELOMERE DNA QUADRUPLEX STRUCTURE IN K+ SOLUTION HYBRID-2 FORM
COMPND    MOL_ID: 1;
COMPND   2 MOLECULE: DNA (26-MER);
COMPND   3 CHAIN: A;
COMPND   4 ENGINEERED: YES
 ObjectMolecule: Read crystal symmetry information.
 ObjectMoleculeReadPDBStr: read MODEL 1
 ObjectMoleculeReadPDBStr: read MODEL 2
 ObjectMoleculeReadPDBStr: read MODEL 3
 ObjectMoleculeReadPDBStr: read MODEL 4
 ObjectMoleculeReadPDBStr: read MODEL 5
 ObjectMoleculeReadPDBStr: read MODEL 6
 ObjectMoleculeReadPDBStr: read MODEL 7
 ObjectMoleculeReadPDBStr: read MODEL 8
 ObjectMoleculeReadPDBStr: read MODEL 9
 ObjectMoleculeReadPDBStr: read MODEL 10
 CmdLoad: "" loaded as "2JPZ".
PyMOL>load K.cif
 ExecutiveLoad-Detail: Detected che

## 3. Transfer rcsb format to charmm format

In [10]:
## transfer to charmm format
!bash {rcsb2charmmformat_sh} {pdbid}_dna.pdb > {pdbid}_dna.0.pdb
!bash {rcsb2charmmformat_sh} {pdbid}_ion.pdb > {pdbid}_ion.0.pdb

## 4. Setup Dna via charmm

In [11]:
charmm_script = f'''\
* Author: Shih-Hsueh Hsu (Allen)
* Title: Generate the G-quadruplex (PDBID:{pdbid}) sturcture 
*        and export to .pdb file
*

set curdir = .
set topdir = {toppar}
set pdbdir = .


set dna = {pdbid}
set dna_part = dna
set ion_part = ion

open unit 11 read form name @topdir/top_all36_na.rtf
read rtf card unit 11
close unit 11

open read unit 11 card name @topdir/par_all36_na.prm
read param unit 11 flex
close unit 11

open read unit 11 card name @topdir/toppar_water_ions.str
stream card unit 11
close unit 11

read sequ card
*  
   26
!TTAGGGTTAGGGTTAGGGTTAGGGTT
THY THY
ADE GUA GUA GUA THY THY ADE GUA GUA GUA 
THY THY ADE GUA GUA GUA THY THY ADE GUA 
GUA GUA THY THY
generate dna setup first 5ter last 3ter

! (default)RNA -> DNA 
patch deo5 dna  1 setup warn
patch deox dna  2 setup warn
patch deox dna  3 setup warn
patch deox dna  4 setup warn
patch deox dna  5 setup warn
patch deox dna  6 setup warn
patch deox dna  7 setup warn
patch deox dna  8 setup warn
patch deox dna  9 setup warn
patch deox dna 10 setup warn
patch deox dna 11 setup warn
patch deox dna 12 setup warn
patch deox dna 13 setup warn
patch deox dna 14 setup warn
patch deox dna 15 setup warn
patch deox dna 16 setup warn
patch deox dna 17 setup warn
patch deox dna 18 setup warn
patch deox dna 19 setup warn
patch deox dna 20 setup warn
patch deox dna 21 setup warn
patch deox dna 22 setup warn
patch deox dna 23 setup warn
patch deox dna 24 setup warn
patch deox dna 25 setup warn
patch deox dna 26 setup warn 

open read unit 12 card name @curdir/@{{dna}}_@{{dna_part}}.0.pdb
read coor pdb unit 12
close unit 12

read sequ card
*  
   2
POT POT
generate ion setup

open read unit 12 card name @curdir/@{{dna}}_@{{ion_part}}.0.pdb
read coor pdb unit 12
close unit 12

print coor

define fix select initialized .and. .not. hydrogen show end

define nonH select .not. hydrogen end
set nonHatoms = ?nsel
define missing select .not. hydrogen .and. .not. initialized show end
set missingatoms = ?nsel

Calc fraction = (@nonHatoms - @missingatoms) / @nonHatoms


! Use of AUTOGENERATE is essential
autogenerate angles dihedrals 

! Internal Coordinate Manipulation
ic fill preserve  !fill: autocomplete hydrogen
ic param
ic build

scaler wmain set 0.0 sele all end
scaler wmain set 1.0 sele fix end

cons fix sele fix end

mini sd   nstep 100
mini abnr nstep 1000
mini abnr nstep 5000

open write unit 12 card name @curdir/@{{dna}}.aa.pdb
write coor pdb unit 12
close unit 12

stop


'''
!echo "{charmm_script}" > {pdbid}.inp

In [12]:
!{charmm_c43b1} < {pdbid}.inp > {pdbid}.dat

### 5. Transfer charmm format to rcsb format

In [13]:
!bash {charmm2rcsbformat_sh} {pdbid}.aa.pdb > {pdbid}.pgro.pdb

### (optinal) 6. Visulize via PyMol

In [14]:
!{pymol} {pdbid}.pgro.pdb

 PyMOL(TM) 3.0.3 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
 License Expiry date: 01-mar-2025
 Detected OpenGL version 4.6. Shaders available.
 Detected GLSL version 4.60.
 OpenGL graphics engine:
  GL_VENDOR:   NVIDIA Corporation
  GL_RENDERER: NVIDIA GeForce GTX 1650/PCIe/SSE2
  GL_VERSION:  4.6.0 NVIDIA 535.183.01
 Detected 12 CPU cores.  Enabled multithreaded rendering.
 CmdLoad: "" loaded as "2jpz.pgro".
